In [3]:
import json
import os
import random
import re
import unicodedata
import sys
from dateutil import parser

import numpy as np
import pandas as pd
import unirest

sys.path.insert(0, '../archive')
import processing

In [4]:
tweets_data = []
data = 'rawTweets.txt'
tweets_file = open(data, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [9]:
#extracting text
data = []
for item in tweets_data:
    data.append(item.get("text"))
    
#processed the tweets to remove extra characters
processed = []
for tweet in data:
    try:
        processed.append(processing.processTweet(tweet))
    except:
        continue
        
#format the tweet to remove AT_USER and URL and format from unicode text
formatted = [re.sub("^rt|AT_USER|URL", "", unicodedata.normalize('NFKD', item).encode('ascii','ignore')) for item in processed]

#extract time of tweets
times = []
for item in tweets_data:
    times.append(item.get("created_at"))
    
#convert the lists to a dataframe
data = pd.DataFrame([times, formatted])
data = data.transpose()
data.columns = ['times','tweet']
data['tweet'] = map(lambda x: str(x).strip(), data['tweet'])
data.head()

,times,tweet
0,Fri Jan 29 01:45:03 +0000 2016,are you rooting for in tonight's gopdebate? r...
1,Fri Jan 29 01:45:03 +0000 2016,a2: :ast gopdebate featured trump humanizing h...
2,Fri Jan 29 01:45:03 +0000 2016,.agree...disgusting a supposed christian hucka...
3,Fri Jan 29 01:45:04 +0000 2016,"how's that milkshake, ? gopdebate"
4,Fri Jan 29 01:45:04 +0000 2016,are you rooting for in tonight's gopdebate? r...


In [10]:
data.to_csv('tweets.csv', index = False)

In [13]:
sentiment = []
for text in data['tweet']:
    try:
    response = unirest.post("https://twinword-sentiment-analysis.p.mashape.com/analyze/",
      headers={
        "X-Mashape-Key": os.environ.get('NLP_KEY'),
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json"
      },
      params={
        "text": text
      }
    )
    sentiment.append(response.body.items()[6])

AttributeError: 'str' object has no attribute 'items'

In [21]:
# These code snippets use an open-source library. http://unirest.io/python
sentiment = []
for text in data['tweet']:
    try:
        response = unirest.post("https://community-sentiment.p.mashape.com/text/",
          headers={
            "X-Mashape-Key": "PSzWEGLQ2VmshEOfW2ZT8EigxkjGp1xpEarjsnjlkBjD4dZdMU",
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json"
          },
          params={
            "txt": text
          }
        )
        sentiment.append(response.body.items()[0][1]['sentiment'])
        unirest.timeout(5)
    except:
        sentiment.append('ERROR')
        unirest.timeout(5)
        continue

In [42]:
withSentiment = data.join(pd.DataFrame(sentiment))
withSentiment.columns = ['time','tweet','sentiment']
withSentiment.to_csv('data_with_sentiment.csv', index = False)

In [38]:
sentimentDf.columns = ['sentiment']
sentimentDf.head()

,sentiment
0,Positive
1,Neutral
2,Negative
3,Neutral
4,Positive


In [40]:
sentimentDf['sentiment'].value_counts()

Negative    169936
Neutral     124069
Positive    110112
ERROR           10
Name: sentiment, dtype: int64